In [1]:
# %load_ext autoreload
# %autoreload 2
%matplotlib inline
import numpy as np
import time
import shutil

import warnings
import intake
import pathlib
import xarray as xr
import pandas as pd
import cf_xarray

import matplotlib.pyplot as plt
from fastjmd95 import rho
# does this improve performance? Probably not...

from dask.diagnostics import ProgressBar
import matplotlib.pyplot as plt

from fastprogress.fastprogress import progress_bar

from xarrayutils.file_handling import (
    write,
    maybe_create_folder,
    file_exist_check,
    temp_write_split,
)
from xarrayutils.utils import (
    remove_bottom_values,
    #mask_mixedlayer
)
from cmip6_preprocessing.preprocessing import (
    combined_preprocessing
)
from cmip6_preprocessing.drift_removal import (
    remove_trend,
    match_and_remove_trend
)
from cmip6_preprocessing.utils import (
    cmip6_dataset_id
)

from cmip6_preprocessing.postprocessing import (
    combine_datasets,
    match_metrics,
    merge_variables,
    interpolate_grid_label
)

import sys
sys.path.append("../../")
from cmip6_omz.upstream_stash import (
    transform_wrapper,
    match_and_detrend,
    concat_experiments, 
    pick_first_member
)
from cmip6_omz.omz_tools import (
    omz_thickness,
    sigma_bins,
    align_missing,
    preprocessing_wrapper,
    vol_consistency_check_wrapper
)

from cmip6_omz.utils import (
    cmip6_collection,
    o2_models,
)

from cmip6_omz.plotting import plot_omz_results

## What I have done:
- Remove all old refs to the other repos
- Refactoring of the metrics matching
- Using only the regridding to combine variables 
    - Need to patch in Norwegian models
- Single cell for filtering/checking all datasets for required vars/metrics
    - This also logs all the problems in one place


## TODO:

- Test with netcdf archive (or at least update the zarr?
- Test performance with strip encoding?
- Try with new trends
- Gotta fix the logic in the interpolate function to just merge variables that all have the same grid label (Nor ESM)
- **The damn norwegian models have no area...**

In [2]:
import dask, distributed
print(dask.__version__)
print(distributed.__version__)

2021.07.1
2021.07.1


In [3]:
from distributed import LocalCluster, Client
mem_total = 350
workers = 4
threads = 10

cluster = LocalCluster(
    memory_limit=f"{int(mem_total/workers)}GiB",
    dashboard_address=9999,
    threads_per_worker=threads,
    n_workers = workers,
                      )
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: LocalCluster
Dashboard: http://127.0.0.1:9999/status,
Status: running,Using processes: True
Dashboard: http://127.0.0.1:9999/status,Workers: 4
Total threads: 40,Total memory: 348.00 GiB
Comm: tcp://127.0.0.1:44780,Workers: 4
Dashboard: http://127.0.0.1:9999/status,Total threads: 40
Started: Just now,Total memory: 348.00 GiB
Comm: tcp://127.0.0.1:39845,Total threads: 10
Dashboard: http://127.0.0.1:36775/status,Memory: 87.00 GiB
Nanny: tcp://127.0.0.1:38482,


# Develop functions here

In [4]:
#This could go upstream in a more general form
## but for now let's keep it here and readable


In [5]:
#this function should go to upstream_stash
def load_trend_dict(ds_dict, verbose = False):
    
    path_jb = '/tigress/GEOCLIM/LRGROUP/jbusecke/projects/aguadv_omz_busecke_2021/data/processed/linear_regression_time_zarr_multimember'
    trendfolder = pathlib.Path(path_jb)
    trend_models = np.unique([ds.attrs['source_id'] for ds in ds_dict.values()])
    flist = []
    for tm in trend_models:
        flist = flist + list(trendfolder.glob(f'*{tm}*_trend.nc'))
    
    total = len(flist)
    progress = progress_bar(range(total))
    
    trend_dict = {}
    for i,path in enumerate(flist):
        key = path.stem
        ds = xr.open_mfdataset([path])
        # write the filename in the dataset
        ds.attrs.update({'filepath':str(path)})
        # exclude all nan slopes (why are these there in the first place?)
        if not np.isnan(ds.slope).all():
            trend_dict[key] = ds
        else:
            if verbose:
                print(f"found all nan regression data for {path}")
        progress.update(i)
    progress.update(total)
    
    return trend_dict


#These are fixes so that the trend data works with cmip6_pp match_and_remove_trend
#these issues should be addressed in the next iteration of trend file production
def fix_trend_metadata(trend_dict):
    for name, ds in trend_dict.items():
        #restore attributes to trend datasets using file names
        fn = (ds.attrs['filepath']).rsplit("/")[-1]
        fn_parse = fn.split('_')
        ds.attrs['source_id'] = fn_parse[2]
        ds.attrs['grid_label'] = fn_parse[5]
        ds.attrs['experiment_id'] = fn_parse[3]
        ds.attrs['table_id'] = fn_parse[4]
        ds.attrs['variant_label'] = fn_parse[7]
        ds.attrs['variable_id'] = fn_parse[8]
        
        #rename 'slope' variable to variable_id
        if "slope" in ds.variables:
            ds = ds.rename({"slope":ds.attrs["variable_id"]})
        
        #error was triggered in line 350 of cmip6_preprocessing.drift_removal
        ##this is a temporary workaround, and the one part of this function that might
        ##require an upstream fix (though it might just be an environment issue)
        ds = ds.drop('trend_time_range')
        
        trend_dict[name] = ds
        
    return trend_dict

In [6]:
def resample_yearly(ds_in, freq="1AS"):
    # this drops some coordinates, so i need to convert them to data_vars and then reconvert
    time_coords = [
        co
        for co in list(ds_in.coords)
        if "time" in ds_in[co].dims and co not in ["time", "time_bounds"]
    ]
    ds_out = ds_in.reset_coords(time_coords).coarsen(time=12).mean()
    ds_out = ds_out.assign_coords({co: ds_out[co] for co in time_coords})
    ds_out.attrs.update({k: v for k, v in ds_in.attrs.items() if k not in ["table_id"]})
    return ds_out

In [7]:
def is_zarr(fn):
    extension = fn.split('.')[-1]
    if extension == 'nc':
        is_zarr = False
    elif extension == 'zarr':
        is_zarr = True
    else:
        raise RuntimeError('Unrecognized File Extension')
    return is_zarr

def reload_preexisting(filename, overwrite = True):
    print("Skipping. File exists already.")
    if is_zarr(filename):
        ds_sigma_reloaded = xr.open_zarr(
            filename, use_cftime=True, consolidated=True
        )
    else:
        ds_sigma_reloaded = xr.open_dataset(
            filename, use_cftime = True
        )
        try:
            plot_omz_results(ds_sigma_reloaded)
        except Exception as e:
            print(f"Plotting failed with: {e}")
    return ds_sigma_reloaded
    
def strip_encoding(ds):
    """Strips the encoding from xr.dataset... This seems like a bug to me."""
    for var in ds.variables:
        ds[var].encoding = {}
    ds.encoding = {}
    return ds

### Local convenience functions for final cell

# Start pipeline here

In [8]:
foldername = "fine_density_tests_combined"
# ofolder = maybe_create_folder(f"../../data/external/{foldername}")
ofolder = maybe_create_folder(f"../../data/processed/{foldername}")
tempfolder = maybe_create_folder(f"../../data/temp/scratch_temp/{foldername}")

# global parameters
o2_bins = np.array([10, 40, 60, 80, 100, 120])
fine_sigma_bins = sigma_bins()
fine_sigma_bins

/scratch/gpfs2/jbusecke/conda_tigressdata/envs/cmip6_omz/lib/python3.9/site-packages/xarrayutils/file_handling.py:118: UserWarning: Folder ../../data/processed/fine_density_tests_combined does already exist.
  warnings.warn(f"Folder {path} does already exist.", UserWarning)
/scratch/gpfs2/jbusecke/conda_tigressdata/envs/cmip6_omz/lib/python3.9/site-packages/xarrayutils/file_handling.py:118: UserWarning: Folder ../../data/temp/scratch_temp/fine_density_tests_combined does already exist.
  warnings.warn(f"Folder {path} does already exist.", UserWarning)


array([  0.  ,  22.5 ,  22.75,  23.  ,  23.25,  23.5 ,  23.75,  24.  ,
        24.25,  24.5 ,  24.75,  25.  ,  25.25,  25.5 ,  25.75,  26.  ,
        26.25,  26.5 ,  26.55,  26.6 ,  26.65,  26.7 ,  26.75,  26.8 ,
        26.85,  26.9 ,  26.95,  27.  ,  27.05,  27.1 ,  27.15,  27.2 ,
        27.25,  27.3 ,  27.35,  27.4 ,  27.45,  27.5 ,  27.55,  27.6 ,
        27.65,  27.7 ,  27.75,  27.8 ,  27.85, 100.  ])

In [9]:
# if this does not work on jupyter.rc, we can add some logic to 
col = intake.open_esm_datastore(cmip6_collection(zarr=True)) #TODO: Check with nc files

z_kwargs={"decode_times": True, "use_cftime": True, "consolidated": True}
n_kwargs={"decode_times": True, "use_cftime": True, 'chunks':{'time':3}}

variable_ids = ["thetao", "so", "o2", "agessc"] #"mlotst"
metric_variable_ids = ["thkcello", "areacello"] #"mlotst"

# models = o2_models()
# models = ['CanESM5-CanOE', 'GFDL-ESM4','CanESM5']# # shorter test run....'IPSL-CM6A-LR',
# models = [m for m in o2_models() if 'GFDL-ESM4' in m or 'Nor' in m]
models = [m for m in o2_models() if 'Nor' in m]

cat = col.search(
    source_id = models,
    grid_label=["gr", "gn"],
    experiment_id=["historical", "ssp585"],
    table_id=["Omon"],
    variable_id=variable_ids,
)
ds_dict = cat.to_dataset_dict(
        aggregate=False,
        zarr_kwargs=z_kwargs,
        cdf_kwargs=n_kwargs,
        preprocess=combined_preprocessing,
    )

# make a separate metric dict to catch all possible metrics!
cat_metrics = col.search(source_id=models,variable_id=metric_variable_ids)
ds_metric_dict = cat_metrics.to_dataset_dict(
        aggregate=False,
        zarr_kwargs=z_kwargs,
        cdf_kwargs=n_kwargs,
        preprocess=combined_preprocessing,
    )


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.member_id.table_id.variable_id.grid_label.version.zstore'



--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.member_id.table_id.variable_id.grid_label.version.zstore'


In [11]:
# I had some trouble with the 'bnds' dimension. So let me drop that one to avoid matching problems (due to int/float problems...geez)
ds_dict = {k:ds.drop('bnds') for k,ds in ds_dict.items()}

In [12]:
# trend_dict = load_trend_dict(ds_dict)

# #comment out this line once trend files have attributes
# trend_dict = fix_trend_metadata(trend_dict)

In [13]:
# new files (change in later and get rid of `load_trend_dict` (or refactor?) and `fix_trend_metadata`)
# Load all trend files
flist = list(pathlib.Path('../../data/external/cmip6_control_drifts/').absolute().glob('*.nc'))
flist = [f for f in flist if any([v in str(f) for v in variable_ids])]
trend_dict = {}
for f in progress_bar(flist):
    trend_dict[f.stem] = xr.open_mfdataset([f])

In [15]:
# these ones are messed up...need a better way to deal with that in the previous step
# see https://github.com/jbusecke/cmip6_preprocessing/issues/175
incomplete_keys = ['CMIP.IPSL.IPSL-CM6A-LR.historical.r3i1p1f1.Omon.gn.none.area_o2']
trend_dict = {k:ds for k,ds in trend_dict.items() if k not in incomplete_keys}

In [16]:
# with warnings.catch_warnings():
#     warnings.filterwarnings("ignore") # TODO: I would like ot take this out, just to see what is going on.
ddict_tracers_detrended = match_and_remove_trend(
    ds_dict,
    trend_dict
)

/scratch/gpfs2/jbusecke/conda_tigressdata/envs/cmip6_omz/lib/python3.9/site-packages/cmip6_preprocessing/postprocessing.py:60: UserWarning: Could not find a matching dataset for CMIP.NCC.NorCPM1.historical.r2i1p1f1.Omon.gn.none
  warnings.warn(nomatch_msg)
/scratch/gpfs2/jbusecke/conda_tigressdata/envs/cmip6_omz/lib/python3.9/site-packages/cmip6_preprocessing/postprocessing.py:60: UserWarning: Could not find a matching dataset for CMIP.NCC.NorCPM1.historical.r4i1p1f1.Omon.gn.none
  warnings.warn(nomatch_msg)
/scratch/gpfs2/jbusecke/conda_tigressdata/envs/cmip6_omz/lib/python3.9/site-packages/cmip6_preprocessing/postprocessing.py:60: UserWarning: Could not find a matching dataset for CMIP.NCC.NorCPM1.historical.r22i1p1f1.Omon.gr.none
  warnings.warn(nomatch_msg)
/scratch/gpfs2/jbusecke/conda_tigressdata/envs/cmip6_omz/lib/python3.9/site-packages/cmip6_preprocessing/postprocessing.py:60: UserWarning: Could not find a matching dataset for CMIP.NCC.NorCPM1.historical.r9i1p1f1.Omon.gr.none


## See if files are actually detrended

In [17]:
# detrended_names = []
# for name, ds in ddict_detrended.items():
#     if ds.attrs['variable_id'] == 'o2':
#         ds_detrnd = ddict_detrended[name]
#         ds_orig = ds_dict[name]
#         break
           
# orig_o2 = np.nanmean(ds_orig.o2.isel(x = 100, y = 220).data, axis = 1)[-200:] #just look at last 200 months
# detrnd_o2 = np.nanmean(ds_detrnd.o2.isel(x = 100, y = 220).data, axis = 1)[-200:]

In [18]:
# fig, ax = plt.subplots(1,2,figsize = (15, 4))
# ax[0].plot(np.arange(len(orig_o2)), orig_o2, alpha = 0.5, label = 'original')
# ax[0].plot(np.arange(len(detrnd_o2)), detrnd_o2, alpha = 0.5, label = 'detrended')

# ax[1].plot(np.arange(len(orig_o2)), orig_o2 - detrnd_o2, label = 'recovered slope')

## Match metrics (there are still quite a few missing).

In [19]:
# this one causes problems because the time is not as long as the full data...
problem_keys = [
    #shorter run? Missing beginning?
    'CMIP.CNRM-CERFACS.CNRM-ESM2-1.historical.r6i1p1f2.Omon.so.gn.v20200117./projects/GEOCLIM/LRGROUP/jbusecke/projects/cmip_data_management_princeton/builder/../zarr_conversion/CMIP6/CMIP/CNRM-CERFACS/CNRM-ESM2-1/historical/r6i1p1f2/Omon/so/gn/v20200117/CMIP.CNRM-CERFACS.CNRM-ESM2-1.historical.r6i1p1f2.Omon.so.gn.v20200117.zarr'
]
ddict_tracers_detrended_filtered = {k:ds.squeeze() for k, ds in ddict_tracers_detrended.items() if k not in problem_keys}

In [20]:
for name, ds in ddict_tracers_detrended_filtered.items():
    if 'member_id' in ds.dims:
        print(name)
        if len(ds.member_id)>1:
            print(name)

In [21]:
print('matching metrics\n')
ddict_matched = match_metrics(ddict_tracers_detrended_filtered, ds_metric_dict, ['areacello', 'thkcello'], print_statistics=True)

matching metrics

Processed 27 datasets.
Exact matches:{'areacello': 0, 'thkcello': 0}
Other matches:{'areacello': 0, 'thkcello': 27}
No match found:{'areacello': 27, 'thkcello': 0}


/scratch/gpfs2/jbusecke/conda_tigressdata/envs/cmip6_omz/lib/python3.9/site-packages/cmip6_preprocessing/postprocessing.py:545: UserWarning: No matching metrics found for areacello
  warnings.warn(f"No matching metrics found for {mv}")


Do I need to rechunk here for the high res models?

In [22]:
### do this earlier?
def maybe_rechunk(ds):
    if len(ds.x)>500:
        return ds.chunk({'x':-1, 'y':-1, 'lev':-1, 'time':1})
    else:
        return ds
ddict_matched_rechunked = {k: maybe_rechunk(ds) for k,ds in ddict_matched.items()}

In [23]:
print('interpolate grids\n')
ddict_matched_regrid = interpolate_grid_label(ddict_matched_rechunked, merge_kwargs={'compat':'override'}) # This should be a default soon

interpolate grids



/scratch/gpfs2/jbusecke/conda_tigressdata/envs/cmip6_omz/lib/python3.9/site-packages/cmip6_preprocessing/postprocessing.py:122: UserWarning: CMIP.NCC.NorESM2-LM.historical.r2i1p1f1.Omon.gr.none failed to combine with :Could not find any variable with the target_grid_labelgn. Found these instead: ['gr']
  warnings.warn(f"{cmip6_dataset_id(ds)} failed to combine with :{e}")
/scratch/gpfs2/jbusecke/conda_tigressdata/envs/cmip6_omz/lib/python3.9/site-packages/cmip6_preprocessing/postprocessing.py:122: UserWarning: ScenarioMIP.NCC.NorESM2-LM.ssp585.r1i1p1f1.Omon.gr.none failed to combine with :Could not find any variable with the target_grid_labelgn. Found these instead: ['gr']
  warnings.warn(f"{cmip6_dataset_id(ds)} failed to combine with :{e}")
/scratch/gpfs2/jbusecke/conda_tigressdata/envs/cmip6_omz/lib/python3.9/site-packages/cmip6_preprocessing/postprocessing.py:122: UserWarning: CMIP.NCC.NorESM2-LM.historical.r3i1p1f1.Omon.gr.none failed to combine with :Could not find any variable w

In [24]:
#patch the norwegian model in manually
ddict_patch = merge_variables(ddict_matched)
for name, ds in ddict_patch.items():
    if 'Nor' in name and 'gr' in name:
        patch_name = name.replace('.gr','')
        ddict_matched_regrid[patch_name] = ds

/scratch/gpfs2/jbusecke/conda_tigressdata/envs/cmip6_omz/lib/python3.9/site-packages/cmip6_preprocessing/postprocessing.py:122: UserWarning: CMIP.NCC.NorESM2-MM.historical.r2i1p1f1.Omon.gr.none failed to combine with :indexes along dimension 'time' are not equal
  warnings.warn(f"{cmip6_dataset_id(ds)} failed to combine with :{e}")


In [25]:
np.sort(list(ddict_matched_regrid.keys()))

array(['NorESM2-LM.historical.Omon.r1i1p1f1',
       'NorESM2-LM.historical.Omon.r2i1p1f1',
       'NorESM2-LM.historical.Omon.r3i1p1f1',
       'NorESM2-LM.ssp585.Omon.r1i1p1f1',
       'NorESM2-MM.historical.Omon.r1i1p1f1',
       'NorESM2-MM.ssp585.Omon.r1i1p1f1'], dtype='<U35')

## Concatenate experiments and pick the first full one

In [26]:
# somehow xarray cannot deal with comparing list/int attrs (Occurs in CM4)
# I should raise that, but lets fix it quickly here
def clean_attrs(ds):
    for a, attr in ds.attrs.items():
        if isinstance(attr, int):
            ds.attrs[a] = [attr]
    return ds

ddict_matched_regrid = {k:clean_attrs(ds) for k, ds in ddict_matched_regrid.items()}

In [27]:
ddict_ex_combined = concat_experiments(ddict_matched_regrid, exclude_attrs=['grid_label', 'variable_id', 'experiment_id'])
ddict_final = pick_first_member(ddict_ex_combined)#

In [28]:
list(np.sort(list(ddict_ex_combined.keys())))

['NorESM2-LM.Omon.r1i1p1f1',
 'NorESM2-LM.Omon.r2i1p1f1',
 'NorESM2-LM.Omon.r3i1p1f1',
 'NorESM2-MM.Omon.r1i1p1f1']

## Check datasets for completeness and log the ones with problems

In [30]:
from cmip6_preprocessing.grids import combine_staggered_grid
problems = {'missing_variables':[], 'missing_area':[], 'missing_thickness':[], 'reconstructed_area':[], 'reconstructed_thickness':[]}
ddict_filtered = {}
for name, ds in ddict_final.items():
    flag = False
    # Check that all necessary variables are given
    missing_variables = [va for va in ["thetao", "so", "o2"] if va not in ds.variables]
    if len(missing_variables)>0:
        flag = True
        problems['missing_variables'].append((name, missing_variables))
        
    # Check for area
    if not 'areacello' in ds.coords:
        if ds.attrs['grid_label'] == 'gr': # only reconstruct for regular grids
            grid, ds = combine_staggered_grid(ds, recalculate_metrics=True)
            # I am dropping dz_t here so it can be uniformly reconstructed
            ds = ds.drop('dz_t')
            ds['areacello'] = ds.dx_t * ds.dy_t
            problems['reconstructed_area'].append(name)
        else:
            flag = True
            problems['missing_area'].append(name)
    
    # Check for thickness (and rename) TODO: We should probably not rename and just refactor to use `thkcello`
    if "thkcello" in ds.coords:
        ds = ds.rename({'thkcello': 'dz_t'})
    else:
        # try to reconstruct the thickness from static info
        try:
            lev_vertices = cf_xarray.bounds_to_vertices(ds.lev_bounds, 'bnds').load()
            dz_t = lev_vertices.diff('lev_vertices')
            ds = ds.assign_coords(dz_t=('lev', dz_t.data))
            problems['reconstructed_thickness'].append(name)
        except Exception as e:
            print(f'{name} thickness reconstruction failed with {e}')
            print(ds)
            problems['missing_thickness'].append(name)
            flag=True
            
    if not flag:
        ddict_filtered[name] = ds

In [31]:
list(np.sort(list(ddict_filtered.keys())))

['NorESM2-LM.gr.Omon', 'NorESM2-MM.gr.Omon']

In [32]:
problems

{'missing_variables': [],
 'missing_area': [],
 'missing_thickness': [],
 'reconstructed_area': ['NorESM2-LM.gr.Omon', 'NorESM2-MM.gr.Omon'],
 'reconstructed_thickness': []}

## The final loop to vertiLocalClustery transform to sigma-space and save output

In [33]:
# overwrite = True
overwrite = False

for mi, (name, ds) in enumerate(ddict_filtered.items()):
    print(f'######################{name} ({mi+1}/{len(ddict_filtered)}) ###############')
    dataset_id = f"{cmip6_dataset_id(ds)}_{ds.attrs['variant_label']}"
    filename = ofolder.joinpath(f"{dataset_id}.zarr")
    t0 = time.time()
    
    

    if file_exist_check(filename) and not overwrite:
        ds_sigma_reloaded = reload_preexisting(filename)
    else:
        print(f"Writing to {filename}")
        tempfilelist = []
        
        # I will have to process the control runs seperately
#         if ds.attrs["experiment_id"] == "piControl":
#             ds = ds.isel(time=slice(-300 * 12, None))

        ds = preprocessing_wrapper(ds)
    
        ds = strip_encoding(ds)

        
        # I need to align.mask the thickness aswell!
        ds = ds.reset_coords(["dz_t"])
        

        #perform nan-masking functions
        print("masking all fields consistently")
        with ProgressBar():
            ds = align_missing(ds)

        print(f"Remove bottom values")
        with ProgressBar():
            ds = remove_bottom_values(ds)
        
        ds = ds.set_coords("dz_t")

        ds["omz_thickness"] = omz_thickness(
            ds, o2_bins=o2_bins
        )
        
        ds["sigma_0"] = (rho(ds.so, ds.thetao, 0) - 1000)
        
        ds = strip_encoding(ds)
        
        # demo plot?

        ds_sigma_monthly = transform_wrapper(ds, sigma_bins=fine_sigma_bins)

        if not vol_consistency_check_wrapper(ds, ds_sigma_monthly):
            continue
        else:
            ds_sigma_yearly = resample_yearly(ds_sigma_monthly)
            
            with ProgressBar():
                ds_sigma_yearly, tempfilelist_var = temp_write_split(
                    ds_sigma_yearly,
                    tempfolder,
                    verbose=False,
                    method='dimension',
                    split_interval=1 if len(ds_sigma_yearly.x)>400 else 10,
                )
            tempfilelist.extend(tempfilelist_var)
            
            # Check metadata
            for ma in ['source_id', 'grid_label', 'table_id', 'variant_label']:
                assert ds.attrs[ma] == ds_sigma_yearly.attrs[ma]

            #################### write out results ########################
            with ProgressBar():
                ds_sigma_reloaded = write(
                    ds_sigma_yearly,
                    filename,
                    overwrite=False,
                    force_load=False,
                    check_zarr_complete=True,
                )
                
            ###### delete temps ######
            print('removing temps')
            for tf in tempfilelist:
                if tf.exists():
                    shutil.rmtree(tf)
        ##################### Verification plotting ##########################
        print('plotting results')
        try:
            plot_omz_results(ds_sigma_reloaded)
        except Exception as e:
            print(f"Plotting failed with: {e}")
    plt.show()
    t1 = time.time()
    print(f"Time passed: {(t1-t0)/60} minutes")

######################NorESM2-LM.gr.Omon (1/2) ###############
Writing to ../../data/processed/fine_density_tests_combined/none.NCC.NorESM2-LM.none.r1i1p1f1.Omon.gr.none_r1i1p1f1.zarr
Replacing intake_esm_varname attrs value with `none`
masking all fields consistently
Remove bottom values
Check if ocean volume is conserved...
Relative difference ocean vol: 8.328428007210887e-11% | OMZ vol [7.04648878e-11 9.24879722e-11 7.33929582e-11 1.66115059e-10
 1.88971590e-10 2.53822322e-10]%


/scratch/gpfs2/jbusecke/conda_tigressdata/envs/cmip6_omz/lib/python3.9/site-packages/dask/core.py:121: RuntimeWarning: invalid value encountered in true_divide
  return func(*(_execute_task(a, cache) for a in args))
/scratch/gpfs2/jbusecke/conda_tigressdata/envs/cmip6_omz/lib/python3.9/site-packages/numba/np/ufunc/gufunc.py:151: RuntimeWarning: invalid value encountered in _interp_1d_linear
  return self.ufunc(*args, **kwargs)
/scratch/gpfs2/jbusecke/conda_tigressdata/envs/cmip6_omz/lib/python3.9/site-packages/dask/core.py:121: RuntimeWarning: invalid value encountered in true_divide
  return func(*(_execute_task(a, cache) for a in args))
/scratch/gpfs2/jbusecke/conda_tigressdata/envs/cmip6_omz/lib/python3.9/site-packages/numba/np/ufunc/gufunc.py:151: RuntimeWarning: invalid value encountered in _interp_1d_linear
  return self.ufunc(*args, **kwargs)
/scratch/gpfs2/jbusecke/conda_tigressdata/envs/cmip6_omz/lib/python3.9/site-packages/dask/core.py:121: RuntimeWarning: invalid value encou

$ Saving 175.358949708GB to ../../data/processed/fine_density_tests_combined/none.NCC.NorESM2-LM.none.r1i1p1f1.Omon.gr.none_r1i1p1f1.zarr
$ Reloading file
removing temps
plotting results


/scratch/gpfs2/jbusecke/conda_tigressdata/envs/cmip6_omz/lib/python3.9/site-packages/xarray/core/indexing.py:1226: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
/scratch/gpfs2/jbusecke/conda_tigressdata/envs/cmip6_omz/lib/python3.9/site-packages/xarray/core/indexing.py:1226: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ... 

Plotting failed with: 'Dataset' object has no attribute 'areacello'
Time passed: 88.59688915014267 minutes
######################NorESM2-MM.gr.Omon (2/2) ###############
Writing to ../../data/processed/fine_density_tests_combined/none.NCC.NorESM2-MM.none.r1i1p1f1.Omon.gr.none_r1i1p1f1.zarr
Replacing intake_esm_varname attrs value with `none`
masking all fields consistently
Remove bottom values
Check if ocean volume is conserved...
Relative difference ocean vol: 8.328492736261351e-11% | OMZ vol [4.58233643e-11 8.23382393e-11 5.60075540e-11 1.91172803e-10
 1.57982580e-10 2.40537858e-10]%


$ Saving 175.358949708GB to ../../data/processed/fine_density_tests_combined/none.NCC.NorESM2-MM.none.r1i1p1f1.Omon.gr.none_r1i1p1f1.zarr
$ Reloading file
removing temps
plotting results
Plotting failed with: 'Dataset' object has no attribute 'areacello'
Time passed: 85.97495584885279 minutes


/scratch/gpfs2/jbusecke/conda_tigressdata/envs/cmip6_omz/lib/python3.9/site-packages/xarray/core/indexing.py:1226: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
/scratch/gpfs2/jbusecke/conda_tigressdata/envs/cmip6_omz/lib/python3.9/site-packages/xarray/core/indexing.py:1226: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ... 